# Overview

- after scraping, scrub the data

# Dependencies

In [1]:
# general
import pandas as pd

# Input

In [2]:
str_output_dir = "./output/example/"

In [3]:
df_us_gaap = pd.read_csv(str_output_dir + "us-gaap.csv")

In [4]:
df_dei = pd.read_csv(str_output_dir + "dei.csv")

In [31]:
# quick check
df_us_gaap.groupby(['CIK']).agg('count')

,Filing,Filing Date,Filing Link,tag,text,contextref,decimals,id,unitref,xsi:nil,xmlns:xsi
CIK,,,,,,,,,,,
789019,18751,18751,18751,18751,18713,18748,17225,16039,17261,38,6
1018724,9822,9822,9822,9822,9787,9822,9458,8391,9489,35,0
1652044,6733,6733,6733,6733,6720,6731,6590,6731,6602,13,0


# US GAAP Tags

I like this explanation of the XLRB in general:<br>
https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
    
It also provides a fantastic list of all of the most important tags!

In [6]:
dict_tags = {
    'us-gaap:accountspayablecurrent': "Liabilities payable to vendors as of the balance sheet date",
    'us-gaap:accountsreceivablegross': "Amounts due from customers or clients",
    'us-gaap:accountsreceivablenet': "Amounts due from customers or clients, reduced to estimated realizable value",
    'us-gaap:accruedincometaxes': "Unpaid sum of known and estimated tax obligations",
    'us-gaap:accruedinsurancecurrent': "Obligations payable to insurance entities to mitigate loss",
    'us-gaap:assetmanagementcosts': "Aggregate costs related to asset management",
    'us-gaap:assets': "Total Assets",
    'us-gaap:assetscurrent': "Sum of all assets expected to be realized within year",
    'us-gaap:borrowedfunds': "Sum of all debt amounts",
    'us-gaap:cash': "Unrestricted cash available for operating needs",
    'us-gaap:commercialpaper': "Value of short-term borrowings using unsecured obligations issued by banks and corporations",
    'us-gaap:commonstocknoparvalue': "Issuance value per share of no-par value stock",
    'us-gaap:commonstocksharesissued': "Total number of common shares that have been sold or granted to shareholders",
    'us-gaap:commonstockvalue': "Aggregate par or stated value of issued common stock",
    'us-gaap:salariesandwages': "Expenditures for salaries other than officers",
    'us-gaap:convertibledebt': "Amount of debt that can be converted into another form of financial instrument, such as common stock",
    'us-gaap:costofgoodssold': "Aggregate costs related to goods sold during the period",
    'us-gaap:costofservices': "Total costs related to services rendered during the period",
    'us-gaap:costsandexpenses': "Total costs of sales and operating expenses for the period",
    'us-gaap:debtcurrent': "Sum of short-term debt and maturities of long-term debt",
    'us-gaap:deferredrevenue': "Cash or other assets that have not yet been realized",
    'us-gaap:depreciation': "Amount of expense related to the cost of tangible assets over the assets' useful lives",
    'us-gaap:directoperatingcosts': "Aggregate expenses directly related to operations",
    'us-gaap:dividends': "Equity impact of cash, stock, and dividends declared  for all securities during the period",
    'us-gaap:earningspersharebasic': "Net income (loss) for the period per share of common stock",
    'us-gaap:grossprofit': "Aggregate revenue minus the cost of goods/services sold and operating expenses",
    'us-gaap:intangibleassetscurrent': "Current portion of non-physical assets, excluding financial assets",
    'us-gaap:interestanddebtexpense': "Expenses related to interest and debt payments",
    'us-gaap:inventorygross': "Merchandise, goods, or supplies held for future sale or used int manufacturing or production",
    'us-gaap:land': "Real estate held for productive use, not held for sale",
    'us-gaap:liabilities': "Sum of all recognized liabilities",
    'us-gaap:liabilitiesandstockholdersequity': "Total of liabilities and stockholder's equity, including the portion of equity attributable to noncontrolling interests",
    'us-gaap:netincomeloss': "Portion of profit or loss for the period, net of income taxes",
    'us-gaap:profitloss': "Consolidated profit or loss for the period",
    'us-gaap:notespayable': "Aggregate amount of notes payable, with initial maturities beyond one year or the normal operating cycle",
    'us-gaap:officerscompensation': "Expenditures for salaries of officers",
    'us-gaap:operatingcycle': "Entity's operating cycle if less than 12 months",
    'us-gaap:operatingexpenses': "Recurring costs associated with normal operations except expenses included in the cost of sales or services",
    'us-gaap:preferredstockvalue': "Stated value of issued nonredeemable preferred stock",
    'us-gaap:researchanddevelopment': "Costs incurred during research and development ",
    'us-gaap:expense': "activities",
    'us-gaap:revenues': "Aggregate revenue recognized during the period",
    'us-gaap:sharesissued': "Number of shares of stock issued",
    'us-gaap:sharesoutstanding': "Number of shares issued and outstanding",
    'us-gaap:stockholdersequity': "Total of stockholders' equity items, net of receivables from officers, directors, owners, and affiliates"
}

# Scrub

Unfortunately, each company uses their own unique contexts!

Assuming we'll be working with AMZN, GOOG, and MSFT often, I invested the manual work of mapping out the contexts for each company.

See the notes below for more info.

In [7]:
def get_text(df, cik, tag, contextref, value = "text"):
    # multiples may exist depending on how it's used in the doc
    try:
        return df[(df['CIK'] == cik) & (df['tag'] == tag) & (df['contextref'] == contextref)].reset_index().iloc[0][value]
    except:
        return None

In [8]:
def get_ls_financials(df, cik, ls_context, ls_tags):
    ls_financials = []
    for tag in ls_tags:
        for contextref in ls_context:
            dict_financials = {
                'CIK': cik,
                'Filing Link': get_text(df, cik, tag, contextref, "Filing Link"),
                'tag': tag,
                'contextref': contextref,
                'text': get_text(df, cik, tag, contextref, "text")
            }
            ls_financials.append(dict_financials)

    return ls_financials

# AMZN

AMZN is consistent. Their contexts go FIYYYYQ4.

In [9]:
# # get df
# ls_financials = get_ls_financials(
#     df_us_gaap, 
#     1018724, 
#     ls_context = ["FI" + str(year) + "Q4" for year in range(2012,2020)], 
#     ls_tags = dict_tags.keys()
# )
# df_amzn_tidy = pd.DataFrame(ls_financials)
# # df_amzn_tidy

In [10]:
# # add FY
# df_amzn_tidy['FY'] = df_amzn_tidy['contextref'].str[2:6]
# df_amzn_tidy

,CIK,Filing Link,tag,contextref,text,FY
0,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:accountspayablecurrent,FI2012Q4,13318000000,2012
1,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:accountspayablecurrent,FI2013Q4,15133000000,2013
2,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:accountspayablecurrent,FI2014Q4,16459000000,2014
3,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:accountspayablecurrent,FI2015Q4,20397000000,2015
4,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:accountspayablecurrent,FI2016Q4,25309000000,2016
...,...,...,...,...,...,...
347,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockholdersequity,FI2015Q4,13384000000,2015
348,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockholdersequity,FI2016Q4,19285000000,2016
349,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockholdersequity,FI2017Q4,27709000000,2017
350,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockholdersequity,FI2018Q4,43549000000,2018


In [36]:
df_amzn_tidy = df_us_gaap[(df_us_gaap['CIK'] == 1018724)][['CIK','Filing Link', 'tag', 'contextref', 'text']]
# df_amzn_tidy['FY'] = df_amzn_tidy['contextref'].str[2:6]
df_amzn_tidy

,CIK,Filing Link,tag,contextref,text
0,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstockparorstatedvaluepershare,FI2018Q4,0.01
1,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstockparorstatedvaluepershare,FI2019Q4,0.01
2,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesauthorized,FI2018Q4,5000000000
3,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesauthorized,FI2019Q4,5000000000
4,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesissued,FI2018Q4,514000000
...,...,...,...,...,...
9817,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockissuedduringperiodvaluestockoptio...,eol_PE7235----0910-K0010_STD_366_20081231_0_41...,624000000
9818,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockissuedduringperiodsharesstockopti...,eol_PE7235----0910-K0010_STD_366_20081231_0_41...,14000000
9819,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:stockrepurchasedduringperiodshares,eol_PE7235----0910-K0010_STD_366_20081231_0_41...,-2000000
9820,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:netincomeloss,eol_PE7235----0910-K0010_STD_366_20081231_0_41...,645000000


# GOOG

GOOG uses the same contexts as AMZN!

In [11]:
# # get df
# ls_financials = get_ls_financials(
#     df_us_gaap, 
#     1652044, 
#     ls_context = ["FI" + str(year) + "Q4" for year in range(2012,2020)], 
#     ls_tags = dict_tags.keys()
# )
# df_goog_tidy = pd.DataFrame(ls_financials)
# # df_goog_tidy

In [12]:
# # add FY
# df_goog_tidy['FY'] = df_goog_tidy['contextref'].str[2:6]

In [37]:
df_goog_tidy = df_us_gaap[(df_us_gaap['CIK'] == 1652044)][['CIK','Filing Link', 'tag', 'contextref', 'text']]
df_goog_tidy

,CIK,Filing Link,tag,contextref,text
9822,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:revenueremainingperformanceobligatione...,NaN,2020-01-01
9823,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:revenueremainingperformanceobligatione...,NaN,2022-01-01
9824,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:allowancefordoubtfulaccountsreceivable...,FI2018Q4,729000000
9825,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:allowancefordoubtfulaccountsreceivable...,FI2019Q4,753000000
9826,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:commercialpaper,FI2018Q4,0
...,...,...,...,...,...
16550,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:fairvalueoffinancialinstrumentspolicy,FD2015Q4YTD,"<div style=""font-family:Times New Roman;font-s..."
16551,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:priorperiodreclassificationadjustmentd...,FD2015Q4YTD,"<div style=""font-family:Times New Roman;font-s..."
16552,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:scheduleofrepurchaseagreements,FD2015Q4YTD,"<div style=""font-family:Times New Roman;font-s..."
16553,1652044,https://www.sec.gov/Archives/edgar/data/165204...,us-gaap:sharebasedcompensationoptionandincenti...,FD2015Q4YTD,"<div style=""font-family:Times New Roman;font-s..."


# MSFT

MSFT has an organized set of contexts too but different from AMZN and GOOG.

MSFT also uses a different fiscal calendar ending 6/30.

In [13]:
# # get df
# ls_financials = get_ls_financials(
#     df_us_gaap, 
#     789019, 
#     ls_context = ["C_0000789019_" + str(year) + "0630" for year in range(2012,2021)], 
#     ls_tags = dict_tags.keys()
# )
# df_msft_tidy = pd.DataFrame(ls_financials)
# # df_msft_tidy

In [14]:
# # add FY
# df_msft_tidy['FY'] = df_msft_tidy['contextref'].str[13:17]

In [38]:
df_msft_tidy = df_us_gaap[(df_us_gaap['CIK'] == 789019)][['CIK','Filing Link', 'tag', 'contextref', 'text']]
df_msft_tidy

,CIK,Filing Link,tag,contextref,text
16555,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:revenueremainingperformanceobligatione...,NaN,2020-07-01
16556,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:accountingstandardsupdateextensiblelist,C_0000789019_20190701_20200630,us-gaap:AccountingStandardsUpdate201601Member
16557,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:accountingstandardsupdateextensiblelist,C_0000789019_20180701_20190630,us-gaap:AccountingStandardsUpdate201601Member
16558,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:accountingstandardsupdateextensiblelist,C_0000789019_20170701_20180630,us-gaap:AccountingStandardsUpdate201601Member
16559,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:accountingstandardsupdateextensiblelist,C_0000789019_20190701_20200630,us-gaap:AccountingStandardsUpdate201601Member
...,...,...,...,...,...
35301,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:netincomeloss,eol_PE8528----1010-K0014_STD_92_20091231_0,6662000000
35302,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:paymentsofdividendscommonstock,eol_PE8528----1010-K0014_STD_92_20091231_0,1139000000
35303,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:revenues,eol_PE8528----1010-K0014_STD_92_20091231_0,19022000000
35304,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:stockrepurchasedduringperiodshares,eol_PE8528----1010-K0014_STD_92_20091231_0,125000000


# Combine & Final Cleanup

## Combine

In [39]:
# combine
df_stacked_financials = pd.concat([df_amzn_tidy, df_goog_tidy, df_msft_tidy], axis = 0, ignore_index = True)
df_stacked_financials

,CIK,Filing Link,tag,contextref,text
0,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstockparorstatedvaluepershare,FI2018Q4,0.01
1,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstockparorstatedvaluepershare,FI2019Q4,0.01
2,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesauthorized,FI2018Q4,5000000000
3,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesauthorized,FI2019Q4,5000000000
4,1018724,https://www.sec.gov/Archives/edgar/data/101872...,us-gaap:commonstocksharesissued,FI2018Q4,514000000
...,...,...,...,...,...
35301,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:netincomeloss,eol_PE8528----1010-K0014_STD_92_20091231_0,6662000000
35302,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:paymentsofdividendscommonstock,eol_PE8528----1010-K0014_STD_92_20091231_0,1139000000
35303,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:revenues,eol_PE8528----1010-K0014_STD_92_20091231_0,19022000000
35304,789019,https://www.sec.gov/Archives/edgar/data/789019...,us-gaap:stockrepurchasedduringperiodshares,eol_PE8528----1010-K0014_STD_92_20091231_0,125000000


## Join on DEI Details

In [16]:
def get_dei_detail(df_dei, filing_link, tag, value = "text"):
    try:
        return df_dei[
            (df_dei['Filing Link'] == filing_link) &
            (df_dei['tag'] == tag)
        ].reset_index().iloc[0][value]
    except:
        return None

In [17]:
ls_dei_tags = [
    "dei:entityregistrantname",
    "dei:tradingsymbol",
    "dei:documentfiscalyearfocus",
    "dei:documentperiodenddate"
]

In [18]:
# get details for all filings
ls_dei_details = []
for filing_link in df_dei['Filing Link'].unique().tolist():
    dict_details = {'Filing Link': filing_link}
    for die_tag in ls_dei_tags:
        dict_details[die_tag[4:]] = get_dei_detail(
            df_dei = df_dei,
            filing_link = filing_link,
            tag = die_tag
        )
    ls_dei_details.append(dict_details)

In [19]:
df_dei_details = pd.DataFrame(ls_dei_details)
df_dei_details

,Filing Link,entityregistrantname,tradingsymbol,documentfiscalyearfocus,documentperiodenddate
0,https://www.sec.gov/Archives/edgar/data/101872...,"AMAZON.COM, INC.",AMZN,2019,2019-12-31
1,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,None,2018,2018-12-31
2,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,None,2017,2017-12-31
3,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2016,2016-12-31
4,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2015,2015-12-31
5,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2014,2014-12-31
6,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2013,2013-12-31
7,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2012,2012-12-31
8,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2011,2011-12-31
9,https://www.sec.gov/Archives/edgar/data/101872...,AMAZON COM INC,AMZN,2010,2010-12-31


In [40]:
df_final = pd.merge(df_stacked_financials, df_dei_details, on = 'Filing Link')

In [21]:
# df_final['tag_def'] = df_final.apply(lambda x: dict_tags[x['tag']], axis = 1)

In [41]:
df_final.to_excel(str_output_dir + "df_final.xlsx", "df_final", index = False, freeze_panes = (1,1))

# Testing

In [27]:
len(df_us_gaap.index)

35306

In [24]:
len(df_dei_details.index)

27

In [42]:
len(df_amzn_tidy.index)

9822

In [43]:
len(df_stacked_financials.index)

35306

In [44]:
len(df_final.index)

35306

In [23]:
# testing
int_index = 0
print(df_final.iloc[int_index]['Filing Link'])
df_final.iloc[int_index]

https://www.sec.gov/Archives/edgar/data/1018724/000101872414000006/0001018724-14-000006-index.htm


CIK                                                                  1018724
Filing Link                https://www.sec.gov/Archives/edgar/data/101872...
tag                                           us-gaap:accountspayablecurrent
contextref                                                          FI2012Q4
text                                                             13318000000
FY                                                                      2012
entityregistrantname                                          AMAZON COM INC
tradingsymbol                                                           AMZN
documentfiscalyearfocus                                                 2013
documentperiodenddate                                             2013-12-31
tag_def                    Liabilities payable to vendors as of the balan...
Name: 0, dtype: object